# Cocoa price forecasting workflow (supervisor-ready)

This notebook follows the workflow outlined in the **WLL Cocoa Extension** proposal (hypothesis section intentionally omitted). Every step is arranged so you can run cells from top to bottom without editing code. The only requirement is that the repository dependencies are installed (see `requirements.txt`).

## 1. Quick start

1. Ensure the virtual environment is active and dependencies are installed: `pip install -r requirements.txt`.
2. Open this notebook (`notebooks/wll_cocoa_extension.ipynb`).
3. Click **Run All** in Jupyter. The notebook will load the processed dataset, train two quick baseline models, and produce plots.

All paths are pre-filled for the checked-in processed dataset (`data/processed/cocoa_ghana_full.csv`).

## 2. Imports and settings

- Adjust `OOS_START_DATE` only if you want to change the out-of-sample window.
- The feature and target lists are defined to match the processed data columns.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from cocoa.models import RFModel, XGBModel

# Plotting style
plt.style.use('seaborn-v0_8')

DATA_PATH = Path('data/processed/cocoa_ghana_full.csv')
# Start date for out-of-sample evaluation
OOS_START_DATE = '2023-01-01'

FEATURE_COLS = [
    'PRCP_anom_mean', 'TAVG_anom_mean', 'PRCP_anom_std', 'TAVG_anom_std',
    'N_stations', 'log_price_lagt', 'log_return'
]
TARGET_COL = 'log_return_forecast_target'


## 3. Load and inspect the processed dataset

The cell below reads the processed Ghana cocoa panel, shows a preview, and reports the coverage period.

In [ ]:
df = pd.read_csv(DATA_PATH, parse_dates=['date'])
print(f"Rows: {len(df):,} | Columns: {list(df.columns)}")
print(df.head())

print(f"
Date range: {df['date'].min().date()} to {df['date'].max().date()}")

## 4. Visualize price levels and returns

Two quick plots help confirm that the data load is working:
- Log price trend
- One-step-ahead log returns (target variable)


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
axes[0].plot(df['date'], df['log_price'], label='Log price', color='navy')
axes[0].set_ylabel('Log price')
axes[0].legend()

axes[1].plot(df['date'], df[TARGET_COL], label='Target log return', color='darkorange')
axes[1].axhline(0, color='gray', linestyle='--', linewidth=1)
axes[1].set_ylabel('Log return target')
axes[1].set_xlabel('Date')
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Train/test split

The split uses a calendar cut: everything before `OOS_START_DATE` is training data; later observations are held out for evaluation.

In [ ]:
train_mask = df['date'] < pd.to_datetime(OOS_START_DATE)
test_mask = ~train_mask

train_df = df.loc[train_mask].copy()
test_df = df.loc[test_mask].copy()

X_train, y_train = train_df[FEATURE_COLS], train_df[TARGET_COL]
X_test, y_test = test_df[FEATURE_COLS], test_df[TARGET_COL]

print(f"Training rows: {len(X_train):,}")
print(f"Test rows: {len(X_test):,}")

## 6. Fit quick baseline models

Two models are run for comparison:
- **Random Forest**: fast nonlinear benchmark.
- **XGBoost**: gradient-boosted trees with modest parameters for quick execution.

Hyperparameters are intentionally lightweight so the notebook remains "click-and-run" friendly.

In [ ]:
rf = RFModel(n_estimators=200, max_features=0.6, min_samples_leaf=2, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

xgb = XGBModel(
    n_estimators=250,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)


## 7. Evaluate and compare

Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) are reported for the out-of-sample period.

In [ ]:
def summarize_metrics(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    return {'model': name, 'MAE': mae, 'RMSE': rmse}

results = [
    summarize_metrics('Random Forest', y_test, rf_pred),
    summarize_metrics('XGBoost', y_test, xgb_pred),
]

metrics_df = pd.DataFrame(results)
print(metrics_df)


## 8. Plot forecasts vs. actuals

The chart below overlays the two model forecasts against the target series for the evaluation window. A horizontal line at zero helps spot direction errors.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(test_df['date'], y_test, label='Actual target', color='black', linewidth=2)
ax.plot(test_df['date'], rf_pred, label='Random Forest forecast', color='teal', alpha=0.8)
ax.plot(test_df['date'], xgb_pred, label='XGBoost forecast', color='crimson', alpha=0.8)
ax.axhline(0, color='gray', linestyle='--', linewidth=1)
ax.set_ylabel('Log return target')
ax.set_xlabel('Date')
ax.legend()
plt.tight_layout()
plt.show()

## 9. Save quick-look outputs (optional)

If you want to keep the evaluation numbers and plot, uncomment the save lines below. Files will be written under `reports/`.


In [ ]:
# Uncomment to save outputs
# output_dir = Path('reports')
# output_dir.mkdir(exist_ok=True)
# metrics_df.to_csv(output_dir / 'wll_click_run_metrics.csv', index=False)
# fig_path = output_dir / 'wll_click_run_forecast.png'
# plt.savefig(fig_path, dpi=200)
# print(f"Saved metrics and plot to {output_dir}")